### Libraries and Paths

In [2]:
import os
import arcpy
import numpy as np
import math
import scipy as sp
import scipy.stats
from matplotlib import pyplot as plt
from datetime import datetime

dirPath = os.path.join('D:\M.Sc. Gesopatial Tecnologies\Courses\SIW011 - Python in GIS\Final Project\White-Goose-Light-Pollution\Data')
inVector = os.path.join(dirPath , 'Vector','points.shp')
biomeVector = os.path.join(dirPath,'Vector','tnc_terr_ecoregions.shp')
inRasterFolder = os.path.join(dirPath,'Images')
outRasterFolder = os.path.join(dirPath,'Images_Extent')

### Clip rasters to goose spatial extent

In [1]:
arcpy.env.workspace = inRasterFolder

## getting extent and adding 1000m
desc = arcpy.Describe(inVector)
extent = desc.extent
bufferDistance = 0.0001388  ##1000m in degree units
extentString = str(extent.XMin - bufferDistance) + ' ' + str(extent.YMin - bufferDistance) + ' ' +  \
                str(extent.XMax + bufferDistance) + ' ' + str(extent.YMax + bufferDistance) 
###clipping each raster
inRasters = arcpy.ListRasters()
for raster in inRasters:
    rasterDesc =  arcpy.Describe(raster)
    rasterName = rasterDesc.name
    outRaster = os.path.join(outRasterFolder, rasterName)
    
    ##delete raster if exits
    if arcpy.Exists(outRaster):
        arcpy.Delete_management(outRaster)
    
    ## clip raster
    arcpy.Clip_management (raster, extentString, outRaster, \
                     maintain_clipping_extent = 'NO_MAINTAIN_EXTENT')

3.6740312 51.1431912 87.6671388 73.5753088


### Calculate initial breeding and speed values

In [3]:
def checkBreedingTime(date):
    for year in range(2006 , 2010):
        breedingStartTime = datetime(year, 5, 24)
        breedingEndTime = datetime(year, 6, 7)
        if date > breedingStartTime and date < breedingEndTime:
            isBreeding = True
            break
        else:
            isBreeding = False
    return isBreeding

In [4]:
def distanceKm(lat1, lon1, lat2, lon2):
    p = math.pi/180
    a = 0.5 - math.cos((lat2-lat1)*p)/2 + math.cos(lat1*p) * math.cos(lat2*p) * (1-math.cos((lon2-lon1)*p))/2
    distance = 2* 6371 * math.asin(math.sqrt(a))
    return distance

In [5]:
###classifying breeding season and adding light, speed and date fields
listFields = [field.name for field in arcpy.ListFields(inVector)]
checkFields = {'isBreed':'SHORT', 'date':'DATE','speed':'DOUBLE', 'NTL_int': 'SHORT'}
for field, fieldType in checkFields.items():
    if field in listFields:
        arcpy.DeleteField_management(inVector, field)
    arcpy.AddField_management(inVector, field, fieldType)

    
fields = ['SHAPE@XY', 'tag_ident', 'timestamp', 'isBreed', 'date', 'speed']

with arcpy.da.UpdateCursor(inVector,fields) as cursor:
    index = 0
    for row in cursor:
        
        # Get current coordinates, time and goose id information 
        currentTime = datetime.strptime(row[2], '%Y-%m-%d %H:%M:%S')
        lon , lat = row[0]
        currentGroup = row[1]
        
        # Calculate values starting from the second feature for each group - white-fronted goose id -
        if index > 0:
            if currentGroup == previousGroup:
                distance = distanceKm(lat, lon, lat2, lon2)
                seconds = abs((currentTime - previousTime).total_seconds())
                speed = distance/seconds if seconds > 0 else 0
            else:
                speed = 0
        else:
            speed = 0
        
        ## updating fields
        row[3] = 0
        row[4] = currentTime.date()
        row[5] = speed
        
        # Set previos information by assigning current information - for next iteration
        previousTime = currentTime
        previousGroup = currentGroup
        lon2 = lon
        lat2 = lat
        
        index = index + 1
        cursor.updateRow(row)

### Identify breeding points based on time and biome

In [2]:
### Identifying breeding biomes
arcpy.MakeFeatureLayer_management(biomeVector, 'biome')
arcpy.SelectLayerByLocation_management('biome', 'INTERSECT', inVector)
arcpy.SelectLayerByAttribute_management('biome', 'SUBSET_SELECTION', " ECO_NAME LIKE '%Tundra%' ")
arcpy.CopyFeatures_management('biome', os.path.join(dirPath, 'tundra_breed.shp'))

tundraVector  = os.path.join(dirPath,'Vector','tundra_breed.shp')

In [6]:
## identify breeding points based on time and location
with arcpy.da.UpdateCursor(inVector,['SHAPE@XY','timestamp','isBreed']) as cursor:
    for row in cursor:
        x,y = row[0]
        time = datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S')
        cursorShp = arcpy.SearchCursor(tundraVector)
        for rowid in cursorShp:
            poly = rowid.Shape
            if poly.contains(arcpy.Point(x,y)) and checkBreedingTime(time):
                row[2] = 1
        cursor.updateRow(row)

In [46]:
inVectorArray = arcpy.da.FeatureClassToNumPyArray(inVector, ('isBreed'))
breedingCount = (inVectorArray['isBreed'] == 1).sum()
nonBreedingCount = (inVectorArray['isBreed'] == 0).sum()

print('Breeding points: ', breedingCount)
print('Non Breeding points: ', nonBreedingCount)

Breeding points:  436
Non Breeding points:  6927


### Calculate Light Values

In [32]:
def getRasterYear(rasters, year):
    for raster in rasters:
        rasterDesc =  arcpy.Describe(raster)
        if str(year) in rasterDesc.name:
            rasterYear = raster
    return rasterYear

In [42]:
##calculating light values at goose locations based on year.
arcpy.env.workspace = outRasterFolder
lightRasters = arcpy.ListRasters()[0:2] 

for year in range(2007,2009):
    raster = getRasterYear(lightRasters, year)
    with arcpy.da.UpdateCursor(inVector,['SHAPE@XY','date','NTL_int']) as cursor:
        for row in cursor:
            if row[1].year == year:
                x, y = row[0]
                coords = str(x) + " " + str(y)
                row[2] = float(arcpy.GetCellValue_management(raster, coords)[0])
            cursor.updateRow(row)

In [142]:
inVectorArray = arcpy.da.FeatureClassToNumPyArray(inVector, ('date','isBreed','NTL_int'))

breedStats = {}
f, axarr = plt.subplots(2, 1, figsize=(3,5))
for year in range(2007,2009):
    breedingLight = inVectorArray[(inVectorArray['date'] > datetime(year,1,1)) & \
                                  (inVectorArray['date'] < datetime(year,12,31)) & \
                                  (inVectorArray['isBreed'] == 1)]['NTL_int']
    
    stats = {'mean': np.mean(breedingLight), 'median': np.median(breedingLight), \
             'var': np.var(breedingLight), 'std:': np.std(breedingLight)}
    breedStats[year] = stats
    
    axarr[year-2007].hist(breedingLight)
    axarr[year-2007].set_title(str(year))

print(breedStats[2007])
print(breedStats[2008])
plt.show()

{'mean': 4.013986013986014, 'median': 4.0, 'var': 0.034769426377817984, 'std:': 0.18646561714648088}
{'mean': 3.6933333333333334, 'median': 3.0, 'var': 3.0126222222222223, 'std:': 1.7356907046539778}


In [ ]:
###boxplot

### Extract high light pollution places

In [34]:
import arcpy
from arcpy.sa import *
arcpy.CheckOutExtension('Spatial')

'CheckedOut'

In [36]:
arcpy.env.workspace = outRasterFolder
lightRasters = arcpy.ListRasters()

for year in range(2008,2009):
    raster = getRasterYear(lightRasters, year)
    inRaster = arcpy.Raster(raster)
    outCon = Con((inRaster > 30) & (inRaster < 64), 1)
    outPolygons = os.path.join(dirPath, 'Vector', 'class_' + str(year) + '.shp')
    highPolution = arcpy.RasterToPolygon_conversion(outCon, outPolygons, "NO_SIMPLIFY", 'VALUE')
    outTable = os.path.join(dirPath, 'Tables', 'table_light_' + str(year) + '.dbf')
    ZonalStatisticsAsTable(highPolution, 'Id', raster, outTable, 'NODATA', 'MEAN')
    print('Done: ', inRaster)

Done:  F162008.v4b_web.avg_vis.tif


### Relative differences in distances and light pollution growth

In [37]:
def getClassYear(layers, year):
    for layer in layers:
        if str(year) in layer:
            layerYear = layer
    return layerYear

In [95]:
##near analysis
arcpy.env.workspace = os.path.join(dirPath, 'Vector')
classLayers = arcpy.ListFeatureClasses('*class*')

for year in range(2007,2009):
    layer = getClassYear(classLayers, year)
    condition = "date > timestamp '{}-12-31' And date < timestamp '{}-01-01'".format(str(year-1),str(year+1))
    selection = arcpy.SelectLayerByAttribute_management(inVector, selection_type="NEW_SELECTION", where_clause=condition, invert_where_clause="")
    outPath = os.path.join(dirPath , 'Vector', 'points_' + str(year))
    outLayer = arcpy.CopyFeatures_management(selection, outPath)
    outTable = os.path.join(dirPath, 'Tables', 'table_Dist_' + str(year) + '.dbf')
    arcpy.analysis.Near(outLayer, layer, method='Geodesic')
    arcpy.Statistics_analysis(outLayer, outTable, [["NEAR_DIST","MIN"]], ["NEAR_FID","isBreed"])

In [106]:
arcpy.env.workspace = os.path.join(dirPath, 'Tables')
lightTables = arcpy.ListTables('*light*')
distTables = arcpy.ListTables('*Dist*')

index = 0
for light, dist in zip(lightTables,distTables):
    table = arcpy.JoinField_management(dist, "NEAR_FID", light, "Id", ["MEAN"])
    tableArray = arcpy.da.FeatureClassToNumPyArray(table, ('MIN_NEAR_D','MEAN', 'isBreed'))
    if index == 0:
        tableLight = tableArray
        index = index + 1
    else:
        tableLight = np.concatenate((tableLight, tableArray) , axis = 0)

In [125]:
fig, axs = plt.subplots(1,2)

breeding = [0,1]
for breed in breeding:
    tbl = tableLight[tableLight['isBreed'] == breed]
    cor , p = scipy.stats.pearsonr(tbl['MIN_NEAR_D'], tbl['MEAN'])
    label = r'$\rho$:' + str(round(cor,2))
    axs[breed].scatter(tbl['MIN_NEAR_D'], tbl['MEAN'], alpha=0.8, c= 'blue', edgecolors='none', s=30, label=label)
    m, b = np.polyfit(tbl['MIN_NEAR_D'], tbl['MEAN'], 1)
    axs[breed].plot(tbl['MIN_NEAR_D'], m * tbl['MIN_NEAR_D'] + b)
    axs[breed].legend(loc=1)

fig.suptitle('Mean Light Value vs Min distance')
plt.show()

### Relative differences in distances and ligh pollution intensity

In [127]:
arcpy.env.workspace = os.path.join(dirPath, 'Vector')
pointLayers = arcpy.ListFeatureClasses('*points_*')

index = 0
for layer in pointLayers:
    pointArray = arcpy.da.FeatureClassToNumPyArray(layer, ('NTL_int','NEAR_DIST','isBreed'))
    if index == 0:
        tableArray = pointArray
        index = index + 1
    else:
        tableArray = np.concatenate((tableArray, pointArray), axis = 0)

In [148]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(9, 4))
for breed in breeding:
    tableBreed = tableArray[tableArray['isBreed'] == breed]
    breedingDistance = tableBreed[tableBreed['isBreed'] == breed]['NEAR_DIST']
    statsDist = {'min': np.min(breedingDistance),'mean': np.mean(breedingDistance), \
                 'max:': np.max(breedingDistance),'median': np.median(breedingDistance), \
                 'std:': np.std(breedingDistance)}
    corrCoef, p = scipy.stats.pearsonr(tableBreed['NEAR_DIST'],tableBreed['NTL_int'])
    axs[0].violinplot(breedingDistance, positions = [breed],showmeans=False, showmedians=True)
    axs[0].set_title('Violin plot')
    axs[1].boxplot(breedingDistance, positions = [breed])
    axs[1].set_title('Box plot')
    print(statsDist, '\n', corrCoef)
    
index = 0
for ax in axs:
    ax.yaxis.grid(True)
    ax.set_xticks([0,1])
    ax.set_xlabel('Season')
    if index == 0:
        ax.set_ylabel('Distances to light pollutant')  
        index = index + 1
plt.setp(axs, xticks=[0,1],
         xticklabels=['breeding', 'non-breeding'])
plt.show()

{'min': 0.0, 'mean': 105087.51624876991, 'max:': 461343.123834, 'median': 55475.0824789, 'std:': 116081.61220253869} 
 -0.24623900322044084
{'min': 3696.97283464, 'mean': 112416.36667117193, 'max:': 272559.769718, 'median': 59019.61344605, 'std:': 88548.80222212282} 
 -0.30455007680256


In [ ]:
###box plot, merge all the groups
## connect tundra and pollutant
### related to tundrs how far to light
### compared to other places tundra (optional)
### they avoid light or they like tundra?